In this notebook I've tried to adapt some popular solutions and also added some fixes, so it all works on new version of python.

Also, you can notice that there are few refactorings and also some new implementations, so feel free to copy and experiment on your own

In [1]:
import os

import numpy as np
import pandas as pd

import pandas as pd
from matplotlib import pyplot as plt

from fastai import *
from fastai.vision import *

import json

import glob

import cv2
import torch
from tqdm import tqdm_notebook
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from sklearn.model_selection import train_test_split
from torchvision import transforms
from skimage import io
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline

In [2]:
def kaggle_commit_logger(str_to_log, need_print = True):
    if need_print:
        print(str_to_log)
    os.system(f'echo {str_to_log}')

In [4]:
import json
with open(r'/kaggle/input/iwildcam-2020-fgvc7/iwildcam2020_train_annotations.json') as json_file:
    train_data = json.load(json_file)

In [5]:
df_train = pd.DataFrame({'id': [item['id'] for item in train_data['annotations']],
                                'category_id': [item['category_id'] for item in train_data['annotations']],
                                'image_id': [item['image_id'] for item in train_data['annotations']],
                                'file_name': [item['file_name'] for item in train_data['images']]})

df_train.head()

,id,category_id,image_id,file_name
0,a292dd3c-21bc-11ea-a13a-137349068a90,73,96b00332-21bc-11ea-a13a-137349068a90,96b00332-21bc-11ea-a13a-137349068a90.jpg
1,a0afcfc0-21bc-11ea-a13a-137349068a90,4,879d74d8-21bc-11ea-a13a-137349068a90,879d74d8-21bc-11ea-a13a-137349068a90.jpg
2,a306e9c0-21bc-11ea-a13a-137349068a90,227,9017f7aa-21bc-11ea-a13a-137349068a90,9017f7aa-21bc-11ea-a13a-137349068a90.jpg
3,9eed94c4-21bc-11ea-a13a-137349068a90,250,90d93c58-21bc-11ea-a13a-137349068a90,90d93c58-21bc-11ea-a13a-137349068a90.jpg
4,a2a4dd7a-21bc-11ea-a13a-137349068a90,2,887cd0ec-21bc-11ea-a13a-137349068a90,887cd0ec-21bc-11ea-a13a-137349068a90.jpg


In [6]:
df_train.shape

(217959, 4)

In [7]:
df_image = pd.DataFrame.from_records(train_data['images'])

indices = [
    df_train[df_train['image_id'] == _id].index
    for _id in df_image[df_image['location'] == 537]['id'].values
]
for the_index in indices:
    df_train = df_train.drop(df_train.index[the_index])

In [8]:
df_train.shape

(217917, 4)

In [ ]:
# Image.open('../input/iwildcam-2020-fgvc7/train/8792549a-21bc-11ea-a13a-137349068a90.jpg')

In [ ]:
%%time

indices = []
from tqdm import tqdm

for i in tqdm(df_train['file_name']):
    try:
        Image.open('/kaggle/input/iwildcam-2020-fgvc7/train/' + i)
    except:        
        print(i)
        df_train.drop(df_train.loc[df_train['file_name']==i].index, inplace=True)

 31%|███       | 66966/217917 [07:41<22:41, 110.91it/s]

In [ ]:
df_train.shape

In [ ]:
with open(r'/kaggle/input/iwildcam-2020-fgvc7/iwildcam2020_test_information.json') as f:
    test_data = json.load(f)

In [ ]:
df_test = pd.DataFrame.from_records(test_data['images'])
df_test.head()

In [ ]:
batch_size = 64
IMG_SIZE = 128

N_EPOCHS = 4

ID_COLNAME = 'file_name'
ANSWER_COLNAME = 'category_id'
TRAIN_IMGS_DIR = r'../input/iwildcam-2020-fgvc7/train/'
TEST_IMGS_DIR = r'../input/iwildcam-2020-fgvc7/test/'

In [ ]:
train_df, test_df = train_test_split(df_train[[ID_COLNAME, ANSWER_COLNAME]],
                                     test_size = 0.25,                                     
                                     shuffle = True
                                    )
train_df.head(5)

Outputing what classes do we have

In [ ]:
CLASSES_TO_USE = df_train['category_id'].unique()
CLASSES_TO_USE

In [ ]:
NUM_CLASSES = len(CLASSES_TO_USE)
NUM_CLASSES

In [ ]:
CLASSMAP = dict(list(zip(CLASSES_TO_USE, range(NUM_CLASSES))))
REVERSE_CLASSMAP = dict([(v, k) for k, v in CLASSMAP.items()])

In [ ]:
model=models.densenet201(pretrained='imagenet')

In [ ]:
# model = models.densenet121(pretrained='imagenet')

In [ ]:
new_head = torch.nn.Linear(model.classifier.in_features, NUM_CLASSES)
model.classifier = new_head

In [ ]:
# model.load_state_dict(torch.load('../input/iwild2020-torch/model'))

In [ ]:
try:
    model.cuda();
except:
    pass

 The code starts by creating two transforms: a resize transform and a toTensor transform.
 
 The resize transform will enlarge the images by 50% and the toTensor transform will convert them from an image format to a tensor format.
 Next, the code creates two normalizers.
 
 The first normalizer will adjust the mean values of each image so that they are all equal.
 
 This is done using the transforms.Normalize() function.
 
 The second normalizer will adjust the standard deviation values of each image so that they are all equal.
 
 This is done using the transforms.Compose() function, which combines two other transforms together.
 
 Finally, the code creates two trains_augmentation transforms, one for adjusting the mean values and one for adjusting the standard deviation values.
 These transformations will combine together to create an overall train_augmentation transformation that adjusts both mean and standard deviation values in order to make all images look similar (normalized).
 
 The code will first resize the input images to their original dimensions (IMG_SIZE x IMG_SIZE).
 
 Next, it will apply the normalizer to each image.
 
 Finally, it will compose the two augmentations together, yielding a final transformation that resizes the images back down to their original size while maintaining their normalized values.

In [ ]:
normalizer = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_augmentation = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.ToTensor(),
    normalizer,
])

val_augmentation = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.ToTensor(),
    normalizer,
])

-   The code starts by creating a dataset object named "IMetDataset".
-   The dataset contains data about images, along with information about the number of classes and the column names (id_colname and answer_colname).
-   The __len__() method returns the size of the dataset.
-   Next, the code gets an item from the dataset by using the idx parameter.
-   This is done by using the index of the row in which the desired item is located (cur_idx_row), and then getting the value for img_id.
-   This value is used to get an image filename (img_path) that will be opened in Image module.
-   If transforms is not None, then this will be applied to img before it is returned as a result.
-   If answer_colname is not specified, then all items in this dataset are returned as images without labels.
-   However, if answer_colname exists, label will be initialized to 1 for each row where it appears in label_dict .
-   Finally, these values are returned as a tuple (img , label) .
-   The code initializes a dataset consisting of data from a file called "images_dir" and the number of classes (n_classes) to be processed.
-   Additionally, the code defines three variables: id_colname, answer_colname, and label_dict.
-   Next, the __len__() function is used to determine the size of the dataset.
-   Finally, various methods are used to access specific data within the dataset.
-   For example, __getitem__() is used to retrieve an image from the dataset based on its identifier (img_id).
-   Similarly, __len__() is used to determine how many images were retrieved.
-   Finally, label values are created for each class in label_dict using torch.zeros().

In [ ]:
class IMetDataset(Dataset):
    
    def __init__(self,
                 df,
                 images_dir,
                 n_classes = NUM_CLASSES,
                 id_colname = ID_COLNAME,
                 answer_colname = ANSWER_COLNAME,
                 label_dict = CLASSMAP,
                 transforms = None
                ):
        self.df = df
        self.images_dir = images_dir
        self.n_classes = n_classes
        self.id_colname = id_colname
        self.answer_colname = answer_colname
        self.label_dict = label_dict
        self.transforms = transforms
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):        
        cur_idx_row = self.df.iloc[idx]
        img_id = cur_idx_row[self.id_colname]
        img_name = img_id # + self.img_ext
        img_path = os.path.join(self.images_dir, img_name)

        img = Image.open(img_path)

        if self.transforms is not None:
            img = self.transforms(img)

        if self.answer_colname is None:
            return img, img_id
        label = torch.zeros((self.n_classes,), dtype=torch.float32)
        label[self.label_dict[cur_idx_row[self.answer_colname]]] = 1.0

        return img, label  

In [ ]:
train_dataset = IMetDataset(train_df, TRAIN_IMGS_DIR, transforms = train_augmentation)
test_dataset = IMetDataset(test_df, TRAIN_IMGS_DIR, transforms = val_augmentation)

In [ ]:
BS = 48

train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BS, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
def cuda(x):
    return x.cuda(non_blocking=True)

-   The code first calculates the fbeta_score for a training set of data (y_true and y_pred) using the beta function.
-   This function takes in two floats as input, and returns a float value that represents how well the predicted value matches the actual value.
-   The fbeta_score is then calculated by taking the mean of precision and recall values, which is then divided by beta2 to get a final score.
-   Precision refers to how often the predicted values match the actual values, while recall refers to how often all of the predictions are true positives (i.e., they correspond to real data).
-   A higher precision and recall means that more accurate predictions were made, while a lower precision and recall means that there were more false positives (predictions that matched data but was not actually correct).
-   The code also calculates beta2, which is simply 1/beta.
-   This number helps to adjust for bias in predictions when comparing different models or datasets.
-   For example, if you have two models that predict outcomes differently based on some input variable(s), having a higher beta would make those models more similar so that their predictions would be more accurate overall.
-   The code calculates the fbeta_score for a data set of y_true and y_pred values.
-   The fbeta_score function takes in three arguments: y_true, y_pred, and beta.
-   The beta argument is used to adjust the weighting of the predictions made by y_pred relative to the true value.
-   The threshold argument sets a cutoff value for how much more likely a prediction is to be correct than not.
-   Finally, the eps parameter sets the error rate that should be used when calculating the precision and recall values.
-   The precision and recall values are then calculated using these formulas: precision = true_positive.div(y_pred.sum(dim=1).add(eps))

In [ ]:
def f1_score(y_true, y_pred, threshold=0.5):
    return fbeta_score(y_true, y_pred, 1, threshold)


def fbeta_score(y_true, y_pred, beta, threshold, eps=1e-9):
    beta2 = beta**2

    y_pred = torch.ge(y_pred.float(), threshold).float()
    y_true = y_true.float()

    true_positive = (y_pred * y_true).sum(dim=1)
    precision = true_positive.div(y_pred.sum(dim=1).add(eps))
    recall = true_positive.div(y_true.sum(dim=1).add(eps))

    return torch.mean(
        (precision*recall).
        div(precision.mul(beta2) + recall + eps).
        mul(1 + beta2))

-   The code starts by initializing the model.
-   This is done by calling the train() function.
-   The code then loops through each step in the training process, which consists of loading a data set and training the model on it.
-   Each step in the training process involves loading features from a data set and assigning them to variables named features and targets .
-   Then, the loss function is applied to these variables to calculate the overall loss for that step.
-   Finally, an optimization algorithm is used to improve this loss value.
-   If there are more steps left in the training process, then the code updates a log file called kaggle_commit_logger .
-   This file contains information about what was done during that particular step of training (e.g., how much loss was incurred), as well as whether or not print output should be generated.
-   The code will: -Initialize the model.
-   -Start training the model on a data set.
-   -Calculate the loss for each step of the training process.
-   -If there are more steps to be processed, then the code will continue to train on those additional steps.
-   -Once all steps have been processed, it will write out a log message detailing the loss and commit that information to a Kaggle commit logger.

In [ ]:
import contextlib
def train_one_epoch(model, train_loader, criterion, optimizer, steps_upd_logging = 250):
    model.train();

    total_loss = 0.0

    train_tqdm = tqdm_notebook(train_loader)


    for step, (features, targets) in enumerate(train_tqdm):
        with contextlib.suppress(Exception):
            features, targets = cuda(features), cuda(targets)

            optimizer.zero_grad()

            logits = model(features)

            loss = criterion(logits, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            if (step + 1) % steps_upd_logging == 0:
                logstr = f'Train loss on step {step + 1} was {round(total_loss / (step + 1), 5)}'
                train_tqdm.set_description(logstr)
                kaggle_commit_logger(logstr, need_print=False)

    return total_loss / (step + 1)

-   The code begins by initializing some variables.
-   The model variable is a function that will be used to evaluate the data.
-   The valid_loader variable is a function that will be used to load the training data into memory.
-   Next, the code creates an iterator over the training data.
-   This iterator will be used to load each feature and target into memory.
-   The next step in the code is to calculate the loss for each logistic regression model using the features and targets stored in memory from Step 1.
-   The final step in this code is to create a list of all true positives (targets) and all false positives (features).
-   Then, it calculates the mean f1 score for these two lists using torch.cat().
-   Finally, it prints out the results of this calculation onscreen using kaggle_commit_logger().
-   The code first validates the model using the valid_loader.
-   This validator ensures that all of the data in the validation set is of the correct type (i.e.
-   numeric).
-   Next, it creates a test_loss variable to track how well the model performs on this validation set.
-   Finally, it creates a list of targets and logs their corresponding logits values.
-   Next, the code calculates the mean f1 score for all of these targets by taking their sum and dividing it by the number of items in each list.
-   It then prints this value to a logstr variable for later use.
-   Finally, it returns the test_loss value as well as its mean f1 score.

In [ ]:
def validate(model, valid_loader, criterion, need_tqdm = False):
    model.eval();

    test_loss = 0.0
    TH_TO_ACC = 0.5

    true_ans_list = []
    preds_cat = []

    with torch.no_grad():
        
        valid_iterator = tqdm_notebook(valid_loader) if need_tqdm else valid_loader
        for step, (features, targets) in enumerate(valid_iterator):
            features, targets = cuda(features), cuda(targets)

            logits = model(features)
            loss = criterion(logits, targets)

            test_loss += loss.item()
            true_ans_list.append(targets)
            preds_cat.append(torch.sigmoid(logits))

        all_true_ans = torch.cat(true_ans_list)
        all_preds = torch.cat(preds_cat)

        f1_eval = f1_score(all_true_ans, all_preds).item()

    logstr = f'Mean val f1: {round(f1_eval, 5)}'
    kaggle_commit_logger(logstr)
    return test_loss / (step + 1), f1_eval

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
sheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5)

-   The code starts by creating a few variables.
-   The first is train_losses, which will store the average loss for each epoch of training.
-   The second is valid_losses, which will store the average validation loss.
-   The third is valid_f1s, which will store the average F1 score for each epoch of training.
-   Next, the code begins to run the kaggle competition algorithm.
-   First, it creates a commit log message that describes what happened during this particular epoch of training (epoch = 1 in this case).
-   Then it calls the train_one_epoch() function to start training on the model using the test data and criterion specified in train_loader .
-   The next line calculates mean train loss for this particular epoch.
-   This value is stored in tr_loss .
-   Finally, the commit log message is created and stored in tr_loss_logstr .
-   Next, the code runs validate() on model and test data to check if they are equal.
-   If they are not equal then an error occurs and sheduler prints out an error message.
-   If they are equal then valid_loss , valid_f1 , and val_loss_logstr are updated with values from validate().
-   Finally, step
-   The code will: -Start at epoch 1 and keep track of the mean train loss for each epoch.
-   -If the train loss is greater than the best model's f1 at any given epoch, then set the best model to be that particular epoch's model and set best_model_ep to that epoch.

In [ ]:
%%time

TRAIN_LOGGING_EACH = 500

train_losses = []
valid_losses = []
valid_f1s = []
best_model_f1 = 0.0
best_model = None
best_model_ep = 0

for epoch in range(1, N_EPOCHS + 1):
    ep_logstr = f"Starting {epoch} epoch..."
    kaggle_commit_logger(ep_logstr)
    tr_loss = train_one_epoch(model, train_loader, criterion, optimizer, TRAIN_LOGGING_EACH)
    train_losses.append(tr_loss)
    tr_loss_logstr = f'Mean train loss: {round(tr_loss,5)}'
    kaggle_commit_logger(tr_loss_logstr)

    valid_loss, valid_f1 = validate(model, test_loader, criterion)  
    valid_losses.append(valid_loss)    
    valid_f1s.append(valid_f1)       
    val_loss_logstr = f'Mean valid loss: {round(valid_loss,5)}'
    kaggle_commit_logger(val_loss_logstr)
    sheduler.step(valid_loss)

    if valid_f1 >= best_model_f1:    
        best_model = model        
        best_model_f1 = valid_f1        
        best_model_ep = epoch        

In [ ]:
torch.save(best_model.state_dict(), 'model')

In [ ]:
bestmodel_logstr = f'Best f1 is {round(best_model_f1, 5)} on epoch {best_model_ep}'
kaggle_commit_logger(bestmodel_logstr)

In [ ]:
xs = list(range(1, len(train_losses) + 1))

plt.plot(xs, train_losses, label = 'Train loss');
#plt.plot(xs, valid_losses, label = 'Val loss');
plt.plot(xs, valid_f1s, label = 'Val f1');
plt.legend();
plt.xticks(xs);
plt.xlabel('Epochs');

In [ ]:
SAMPLE_SUBMISSION_DF = pd.read_csv(r'../input/iwildcam-2020-fgvc7/sample_submission.csv')
SAMPLE_SUBMISSION_DF.head()

In [ ]:
SAMPLE_SUBMISSION_DF.rename(columns={'Id':'file_name','Category':'category_id'}, inplace=True)
SAMPLE_SUBMISSION_DF['file_name'] = SAMPLE_SUBMISSION_DF['file_name'] + '.jpg'
SAMPLE_SUBMISSION_DF.head()

In [ ]:
subm_dataset = IMetDataset(SAMPLE_SUBMISSION_DF,
                           TEST_IMGS_DIR,
                           transforms = val_augmentation,
                           answer_colname=None
                          )

SUMB_BS = 48

subm_dataloader = DataLoader(subm_dataset,
                             batch_size=SUMB_BS,
                             shuffle=False,
                             pin_memory=True)

-   The code starts by loading the model into memory.
-   Next, the code creates a list of predicates called "preds_cat".
-   This list will store all of the predictions made by the model.
-   The next step is to find the maximum value in this list for each dimension (1).
-   This is done using torch.cat().
-   The first argument is a list of lists and the second argument is a function that takes two arguments: first, a list of values, and second, an integer specifying how many dimensions to consider.
-   In this case, we are only considering one dimension (the number of features).
-   The result of this function is an int which represents the maximum value in that dimension.
-   Finally, we convert this int into a numpy array so that we can work with it more easily.
-   Next, we create another list called "all_preds".
-   This list will contain all of the predictions made by the model.
-   We then use torch.argmax() to find the maximum value in this list for each dimension (1).
-   The result of this function is an int which represents the maximum value in that dimension.
-   Finally, we convert this int into a numpy array so that we can work with it more easily.
-   The code first retrieves the model from the Python environment.
-   Next, it creates an empty list called preds_cat and adds all of the predicted values for each feature in the model to the list.
-   Finally, it calculates the maximum value for all of the predictions in preds_cat and stores that value in all_preds.

In [ ]:
def get_subm_answers(model, subm_dataloader, need_tqdm = False):
    model.eval();
    preds_cat = []
    ids = []

    with torch.no_grad():
        
        if need_tqdm:
            subm_iterator = tqdm_notebook(subm_dataloader)
        else:
            subm_iterator = subm_dataloader

        for features, subm_ids in subm_iterator:
            features = cuda(features)

            logits = model(features)
            preds_cat.append(torch.sigmoid(logits))
            ids += subm_ids

        all_preds = torch.cat(preds_cat)
        all_preds = torch.argmax(all_preds, dim=1).int().cpu().numpy()
    return all_preds, ids

In [ ]:
%%time

best_model.cuda();

subm_preds, submids = get_subm_answers(best_model, subm_dataloader, True)

In [ ]:
len(subm_preds)

-   The code first creates a dictionary of strings to represent the category names and their corresponding ids.
-   It then creates a data frame from this dictionary, renaming the columns as needed.
-   The first column in the data frame is "Category" and it contains the category names.
-   The second column is "Id".
-   Next, the code uses map() to convert each string in Category into an integer.
-   This integer corresponds to the id of that category in the dictionary.
-   Finally, head() prints out the first row of the data frame, which is just "Category" with an id of 1.
-   The code will create a DataFrame named "df_to_process" with the following columns: Category Id

In [ ]:
ans_dict = dict(zip(submids, subm_preds.astype(str)))

df_to_process = (
    pd.DataFrame
    .from_dict(ans_dict, orient='index', columns=['Category'])
    .reset_index()
    .rename({'index':'Id'}, axis=1)    
)
df_to_process['Id'] = df_to_process['Id'].map(lambda x: str(x)[:-4])
df_to_process.head()

In [ ]:
def process_one_id(id_classes_str):
    if id_classes_str:
        return REVERSE_CLASSMAP[int(id_classes_str)]
    else:
        return id_classes_str

In [ ]:
df_to_process['Category'] = df_to_process['Category'].apply(process_one_id)
df_to_process.head()

In [ ]:
df_to_process.to_csv('submission_1.csv', index=False)

# Second variant of solving the problem

In [ ]:
# !pip install tensorflow-gpu==1.14.0 -q
# !pip install keras==2.2.4 -q
# !pip install git+https://github.com/qubvel/efficientnet -q

!pip install tensorflow-gpu -q
!pip install keras -q
!pip install git+https://github.com/qubvel/efficientnet -q

from PIL import Image, ImageDraw
import collections
import glob 
from datetime import datetime as dt
import gc
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import cv2
from copy import deepcopy
import efficientnet.keras as efn 
import glob
from IPython.display import Image
import json
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.applications.densenet import DenseNet201
from keras.applications.resnet_v2 import ResNet50V2
from keras.layers import Dense, Flatten, Activation, Dropout, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, applications
from keras.models import Model, load_model
from keras.callbacks import Callback, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.utils import Sequence
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
from numpy.random import seed
import os
import random
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm

seed(10)


from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(20, 12))
im = Image.open('../input/iwildcam-2020-fgvc7/train/939e25fc-21bc-11ea-a13a-137349068a90.jpg')
plt.imshow(im)

# <div class="h2">Data Overview</div>

In [ ]:
pd.read_csv("../input/iwildcam-2020-fgvc7/sample_submission.csv").head()

## <u>Overview of train & test data</u>

There are color data and gray data.  There seems to be photos taken while it was bright and photos taken at night.

In [ ]:
train_jpeg = glob.glob('../input/iwildcam-2020-fgvc7/train/*')
test_jpeg = glob.glob('../input/iwildcam-2020-fgvc7/test/*')

print("number of train jpeg data:", len(train_jpeg))
print("number of test jpeg data:", len(test_jpeg))

### Train data

In [ ]:
from PIL import Image, ImageDraw

fig = plt.figure(figsize=(25, 16))
for i,im_path in enumerate(train_jpeg[:16]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open(im_path)
    im = im.resize((480,270))
    plt.imshow(im)

In [ ]:
fig = plt.figure(figsize=(25, 16))
for i,im_path in enumerate(train_jpeg[16:32]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open(im_path)
    im = im.resize((480,270))
    plt.imshow(im)

### Test data

In [ ]:
fig = plt.figure(figsize=(25, 16))
for i,im_path in enumerate(test_jpeg[:16]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open(im_path)
    im = im.resize((480,270))
    plt.imshow(im)

## <u>Checking iwildcam2020_megadetector_results.json</u>

This json contains the detection result of the training data by megadetector.

Megadetector is the pre-trained model to detect animals, people, and vehicles in camera trap images.

https://github.com/microsoft/CameraTraps/blob/master/megadetector.md#our-ask-to-megadetector-users

In [ ]:
with open('../input/iwildcam-2020-fgvc7/iwildcam2020_megadetector_results.json', encoding='utf-8') as json_file:
    megadetector_results =json.load(json_file)
    
megadetector_results.keys()

In [ ]:
megadetector_results_df = pd.DataFrame(megadetector_results["images"])
megadetector_results_df.head()

-   Next, it creates a list of detection objects.
-   For each detection object in the list, the code calculates its bbox (bounding box) values.
-   The x1, y1, w_box, and h_box values represent the leftmost corner of the detection's bounding box at (x1, y1), (w_box, h_box), and (xmax-xmin+w_box, ymax-ymin+h_box).
-   Next, the code draws a red line between these four points using ImageDraw.Line().
-   The width parameter specifies how wide the line should be; in this case it is set to 4 pixels.
-   The fill parameter specifies what color to use for the line; in this case it is set to 'Red'.
-   Finally, the code sets imageWidth and imageHeight to their respective size values and prints them out onscreen.
-   The code will create a rectangular bounding box around each detected object in the input image.
-   The code then uses the line() function to draw a red line connecting the four corners of the box.

In [ ]:
#Refered: https://www.kaggle.com/qinhui1999/how-to-use-bbox-for-iwildcam-2020 

def draw_bboxs(detections_list, im):
    """
    detections_list: list of set includes bbox.
    im: image read by Pillow.
    """
    
    for detection in detections_list:
        x1, y1,w_box, h_box = detection["bbox"]
        ymin,xmin,ymax, xmax=y1, x1, y1 + h_box, x1 + w_box
        draw = ImageDraw.Draw(im)
        
        imageWidth=im.size[0]
        imageHeight= im.size[1]
        (left, right, top, bottom) = (xmin * imageWidth, xmax * imageWidth,
                                      ymin * imageHeight, ymax * imageHeight)
        
        draw.line([(left, top), (left, bottom), (right, bottom),
               (right, top), (left, top)], width=4, fill='Red')

In [ ]:
data_num = 8857
im = Image.open("../input/iwildcam-2020-fgvc7/train/" + megadetector_results_df.loc[data_num]['id'] + ".jpg")
im = im.resize((480,270))
draw_bboxs(megadetector_results_df.loc[data_num]['detections'], im)

In [ ]:
plt.imshow(im)

We can also know how many aminals detected by megadetector in each pictures. 

In [ ]:
def get_data(x):
    if x == []:
        return 0
    return len(x)

megadetector_results_df["detected_num"] = megadetector_results_df.loc[:, "detections"].map(get_data)

In [ ]:
fig = plt.figure(figsize=(15, 4))
ax = sns.countplot(x="detected_num", data=megadetector_results_df)
ax.set(ylabel='count')
#ax.set(ylim=(0,80000))
plt.title('distribution of count per animals each data of train')

We can select training data depending on the number of animals in the picture.

In [ ]:
megadetector_results_df = megadetector_results_df[megadetector_results_df['detected_num'] < 2]
megadetector_results_df = megadetector_results_df.rename(columns={'id': 'image_id'})
megadetector_results_df.head()

### Train data

In [ ]:
with open('/kaggle/input/iwildcam-2020-fgvc7/iwildcam2020_train_annotations.json') as json_file:
    train_annotations_json = json.load(json_file)

In [ ]:
train_annotations_json.keys()

In [ ]:
df_annotations = pd.DataFrame(train_annotations_json["annotations"])
df_annotations.head()

In [ ]:
df_images = pd.DataFrame(train_annotations_json["images"])

In [ ]:
df_categories = pd.DataFrame(train_annotations_json["categories"])
df_images.head()

In [ ]:
df_categories.head()

In [ ]:
print(f"There are {len(df_categories) - 1} species")

# I export DataFrame as CSV for convenience. 
df_annotations.to_csv("iwildcam2020_train_annotations_annotations.csv", index=False)
df_images.to_csv("iwildcam2020_train_annotations_images.csv", index=False)
df_categories.to_csv("iwildcam2020_train_annotations_categories.csv", index=False)

### Test data

In [ ]:
with open('/kaggle/input/iwildcam-2020-fgvc7/iwildcam2020_test_information.json') as json_file:
    test_information_json = json.load(json_file)

In [ ]:
test_information_json.keys()

In [ ]:
df_images_test = pd.DataFrame(test_information_json["images"])
df_images_test.head()

In [ ]:
df_categories_test = pd.DataFrame(test_information_json["categories"])
df_categories_test.head()

In [ ]:
# I export DataFrame as CSV for convenience. 
df_images_test.to_csv("iwildcam2020_train_annotations_images_test.csv", index=False)
df_categories_test.to_csv("iwildcam2020_train_annotations_categories_test.csv", index=False)

## <u>Data distributions</u>

I plot train and test data in following perspective:

* Time point

* Category ID

* Location

### When did data taken?

Because animals can change their activity from time to time, we want to understand how data is distributed over time.

**Monthly perspective**

In [ ]:
month_year = df_images['datetime'].map(lambda str: str[2:7])
labels_month_year = sorted(list(set(month_year)))

month_year_test = df_images_test['datetime'].map(lambda str: str[2:7])

Data starts 2013-01 but there seems be some lacks. For example, train data between 2013-11 to 2014-02 are missing. 

Also we can find that train data in between 2013-01 to 2013-07 are rich than other time point.

Train data covers test data in perspective of time point.

In [ ]:
labels_month = sorted(list(set(df_images['datetime'].map(lambda str: str[5:7]))))
labels_month

Train data are bias. In February, March, June and July, data are rich than other months. 

Train data covers test data in perspective of month.

Data for November and December are missing. Do animals hibernate?

**Hourly perspective**

In [ ]:
train_taken_hour = df_images['datetime'].map(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S.%f').hour)
test_taken_hour = df_images_test['datetime'].map(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S.%f').hour)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,7))
ax = plt.subplot(1,2,1)
plt.title('Count of train data per hour')
ax = sns.countplot(train_taken_hour)
ax.set(xlabel='hour', ylabel='count')
ax.set(ylim=(0,20000))

ax = plt.subplot(1,2,2)
plt.title('Count of test data per hour')
ax = sns.countplot(test_taken_hour)
ax.set(xlabel='hour', ylabel='count')
ax.set(ylim=(0,20000))

If we decide arbitrarily during daytime and at night, we can also calculate diurnal and nocturnal data counts.

For example, we define "during daytime" is "6~17 O'clock" and "at night" is "18~5 O'clock",

In [ ]:
train_taken_phase = train_taken_hour.map(lambda x: "daytime" if x >= 6 and x < 18 else "night")
test_taken_phase = test_taken_hour.map(lambda x: "daytime" if x >= 6 and x < 18 else "night")

In [ ]:
#Memory savings
del train_taken_phase
del test_taken_phase
del train_taken_hour
del test_taken_hour
gc.collect()

### How many data are there per animal category Id?

There are a lot of categories in dataset. To confirme how many data are there in each categories, I plot barplot.  

In [ ]:
fig = plt.figure(figsize=(30, 4))
labels_id = sorted(list(set(df_categories["id"])))
ax = sns.barplot(x="id", y="count",data=df_categories, order=labels_id)
ax.set(ylabel='count')
ax.set(ylim=(0,80000))
plt.title('distribution of count per id in train')

In [ ]:
fig = plt.figure(figsize=(30, 4))
labels_id = sorted(list(set(df_categories["id"])))
ax = sns.barplot(x="id", y="count",data=df_categories_test, order=labels_id)
ax.set(ylabel='count')
ax.set(ylim=(0,80000))
plt.title('distribution of count per id in test')

Frequency of data in each id is similar for train and test data.

In [ ]:
fig = plt.figure(figsize=(15, 9))
ax = sns.distplot(df_categories['count'][1:])
ax.set(ylim=(0,0.00005))
ax.set(xlabel='count')
plt.title('distribution of number of data per id zoomed')

Almost data is less than 5000 pictures. 

But we can find that the number of specific IDs is very large and the data is biased.

### How many data per location?

We are required to detect photographs taken at different locations, but how distribute are data  in perspect of  location?

In [ ]:
labels_location_train = sorted(list(set(df_images['location'])))
labels_location_test = sorted(list(set(df_images_test['location'])))
labels_location = labels_location_train + labels_location_test

### Are different animal kinds photographed in different places?

We have location data and animal category data, so we can investigate wheathrer different animal kinds are photographed in different places.

In [ ]:
loc_cat_df_test = pd.merge(df_images, df_annotations, left_on='id', right_on='image_id', how = "inner").loc[:,["location", "category_id", "image_id"]]
loc_cat_df_test.head()

In [ ]:
loc_cat_dict = {}
for loc in set(loc_cat_df_test["location"]):
    loc_cat_dict[loc] = list(set(loc_cat_df_test[loc_cat_df_test["location"] == loc]["category_id"]))   
loc_cat_matrix = np.zeros([loc_cat_df_test["location"].max()+1, loc_cat_df_test["category_id"].max()+1])
loc_cat_matrix.shape

In [ ]:
for loc in loc_cat_dict.keys():
    for cat in loc_cat_dict[loc]:
        loc_cat_matrix[loc, cat] = 1

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = sns.heatmap(loc_cat_matrix)
ax.set(xlabel='category', ylabel='location')
plt.title('Relation between animal categories and locations')

It's a bit hard to see, but columns are the categories of animals and rows are the locations. Since similar patterns appear in the vertical direction, it seems that similar species may appear even in different places.

### How many animals in each photograph sequence?

We can know how many animals each photograph sequence (continuous frame of camera trap from startup to taking) by annotations value of train_annotations_json. Count key seems represent number of animals appear in each photograph sequence. 

Three Pan troglodytes appear in folloing frames.

In [ ]:
df_annotations[2429:2440]

In [ ]:
fig = plt.figure(figsize=(25, 16))
#for i,im_path in enumerate(df_annotations[df_annotations.loc[:,"count"] == 3].loc[:,"image_id"][2429:2440]):
for i,im_path in enumerate(df_annotations.loc[:,"image_id"][2429:2440]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open("../input/iwildcam-2020-fgvc7/train/" + im_path + ".jpg")
    im = im.resize((480,270))
    plt.imshow(im)

Does the number of individuals that appear at one time vary depending on the animal species?

In [ ]:
c = collections.Counter(df_annotations.loc[:,"count"])
animals_in_pict = list(c.keys())
freq = list(c.values())
k = zip(animals_in_pict,freq)

In [ ]:
animals_in_pict_df = pd.DataFrame(sorted(k),columns=['num_of_animals','freq'])

In [ ]:
animals_in_pict_df

Only a limited number of animals are grouped together. 266 species are taken in train set, but only 14 species were photographed, with more than 10 animals at a time. It turns out that there is a relationship between the animal species and the number of individuals reflected.　

In [ ]:
species_morethan_10 = list(set(df_annotations[df_annotations.loc[:,"count"] >= 10].loc[:,"category_id"]))
df_categories[df_categories.loc[:,"id"].isin(species_morethan_10)].loc[:,["id", "name"]]

Species that shows more than 30 individuals at a time in a photo is  only a cow and a boar.

In [ ]:
species_morethan_30 = list(set(df_annotations[df_annotations.loc[:,"count"] >= 30].loc[:,"category_id"]))
df_categories[df_categories.loc[:,"id"].isin(species_morethan_30)].loc[:,["id", "name"]]

In [ ]:
df_annotations[(df_annotations.loc[:,"category_id"] == 2) & (df_annotations.loc[:,"count"] >= 30)].head()

In [ ]:
fig = plt.figure(figsize=(25, 16))
#for i,im_path in enumerate(df_annotations[df_annotations.loc[:,"count"] == 3].loc[:,"image_id"][2429:2440]):
for i,im_path in enumerate(df_annotations.loc[:,"image_id"][149674:149678]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open("../input/iwildcam-2020-fgvc7/train/" + im_path + ".jpg")
    im = im.resize((480,270))
    plt.imshow(im)

In [ ]:
df_annotations[(df_annotations.loc[:,"category_id"] == 71) & (df_annotations.loc[:,"count"] >= 30)].head()

In [ ]:
jpeg ='../input/iwildcam-2020-fgvc7/train/8897a9f8-21bc-11ea-a13a-137349068a90.jpg'
im = Image.open(jpeg)
im = im.resize((480,270))
plt.imshow(im)

# Model example

I prepare EfficientNet example. The point of this model is that I choose photos which one or less　animal is taken for training.

I refered following kernels.

https://www.kaggle.com/ateplyuk/inat2019-starter-keras-efficientnet/data

https://www.kaggle.com/mobassir/keras-efficientnetb2-for-classifying-cloud

In [ ]:
train_anns_df = df_annotations[['image_id','category_id']]
train_img_df = df_images[['id', 'file_name']].rename(columns={'id':'image_id'})
df_train_file_cat = pd.merge(train_img_df, train_anns_df, on='image_id')
df_train_file_cat['category_id']=df_train_file_cat['category_id'].astype(str)
df_train_file_cat.head()

In [ ]:
df_train_file_cat = pd.merge(megadetector_results_df, df_train_file_cat, on='image_id', how='inner')

fig = plt.figure(figsize=(15, 4))
ax = sns.countplot(x="detected_num", data=megadetector_results_df)
ax.set(ylabel='count')
plt.title('distribution of count per animals each data of train')

### Prepare data and model

### Parameters


In [ ]:
batch_size = 128
img_size = 96
lr = 0.001 
nb_classes = 267
nb_epochs = 70

-   The code starts by creating two ImageDataGenerator objects.
-   The first, train_datagen, will be used to generate images of the training data.
-   The second, valid_generator, will be used to generate images of the validation data.
-   Next, the code sets up some parameters for the ImageDataGenerator objects.
-   First, it sets the scale parameter to 1/255 so that the generated images will have a resolution of 256x256 pixels.
-   Next, it sets the validation_split parameter to 0.25 so that 25% of the images in each dataset will be used for validation purposes.
-   Finally, it sets the horizontal_flip flag to True so that all image files will be flipped horizontally (so that they are displayed right-to-left).
-   The next section of code defines how many images should be generated per batch and how many batches should be generated total.
-   The batch size is set to 5000 and the shuffle flag is set to True so that all training data items will be randomly shuffled before being processed by train_datagen.
-   Finally, classes is set to an empty list which means that all training data items will be processed as individual instances rather than being grouped into categories like in a traditional classifier system
-   The code will generate two data sets, one for training and the other for testing.
-   The train_datagen data set will contain images of files with a file name that starts with "train" and has an associated category_id value of 1.
-   The test_datagen data set will contain images of files with a file name that starts with "test" and has an associated category_id value of 2.
-   The code also specifies the dimensions of the generated data sets, as well as how they should be resized.
-   It also specifies whether the images should be flipped horizontally (true) and zoomed in or out (0.3 and 0.3, respectively).
-   Finally, it specifies how much space to shift each image's width

In [ ]:
%%time

train_datagen=ImageDataGenerator(rescale=1./255, 
    validation_split=0.25,
    horizontal_flip = True,    
    vertical_flip=True,
    zoom_range = 0.5,
    width_shift_range = 0.5,
    height_shift_range=0.5
    )

train_generator=train_datagen.flow_from_dataframe(    
    dataframe=df_train_file_cat[:50000],    
    directory="../input/iwildcam-2020-fgvc7/train",
    x_col="file_name",
    y_col="category_id",
    batch_size=batch_size,
    shuffle=True,
    classes = [ str(i) for i in range(nb_classes)],
    class_mode="categorical",    
    target_size=(img_size,img_size))

test_datagen = ImageDataGenerator(rescale=1./255)

valid_generator=test_datagen.flow_from_dataframe(    
    dataframe=df_train_file_cat[50000:],    
    directory="../input/iwildcam-2020-fgvc7/train",
    x_col="file_name",
    y_col="category_id",
    batch_size=batch_size,
    shuffle=True,
    classes = [ str(i) for i in range(nb_classes)],
    class_mode="categorical",  
    target_size=(img_size,img_size))

-   The code first creates an instance of the EfficientNetB3 class.
-   This class is used to create a model that predicts the probability of each class in an image.
-   The weights parameter specifies which features should be used in the model, and include_top determines whether top-level features (such as those at the level of classes) should be included in the prediction.
-   The pooling parameter controls how features are combined when predicting probabilities; by default, they are averaged together.
-   Next, the code sets up a variable called x to store the input data for training and predictions.
-   The output from the EfficientNetB3 class will be stored in x .
-   The next section of code creates a Dense object called predictions .
-   This object will contain all of the predicted probabilities for each image.
-   It is initialized with a size equal to 3 , meaning that it will hold 30 values (one for each pixel in an image).
-   Finally, the get_model() function returns a Model object containing both inputs and predictions .
-   The code first creates an instance of the EfficientNetB3 model.
-   This model is designed to predict the probability of a given class (in this case, images of cats) from a set of training images.
-   Next, the code creates a variable called x which will store the predictions made by the EfficientNetB3 model.
-   Finally, the code uses the Model() function to create a prediction model that can be used to make predictions on new data.

In [ ]:
def get_model():
    K.clear_session()
    base_model =  efn.EfficientNetB3(weights='imagenet', include_top=False, pooling='avg', input_shape=(img_size, img_size, 3))
    x = base_model.output
    predictions = Dense(nb_classes, activation="softmax")(x)
    return Model(inputs=base_model.input, outputs=predictions)

model = get_model()
model.compile(optimizers.Adam(lr=lr, decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=1, mode='auto')

### Training

-   The code starts by importing the necessary libraries.
-   It then creates a model object and sets some properties of that object.
-   The most important property is "steps_per_epoch", which specifies the number of steps per epoch in the training process.
-   The code also sets the "validation_data" and "validation_steps" properties to be equal to the values of "generator" and "2", respectively.
-   Next, the code creates four worker threads and assigns them to the "workers" variable.
-   Finally, it sets up two callbacks: one for early termination (called "early"), and another for reporting progress (called "verbose").
-   The next section of code defines what data will be used during training.
-   First, it imports the validation data set into memory using the Python library pandas.
-   Then, it uses pandas' Series objects to create a dataset that contains all valid predictions made by each model during training.
-   Next, the code calculates how many iterations were completed for each model on each epoch using Pandas' Series objects.
-   This information is stored in an array called "epochs".
-   Finally, it prints out this information to screen so that you can see how well each model is performing overall.
-   The code will run the model on a training set of data and a validation set of data.
-   The model is fit using the generator function, which takes as input a train_generator object and an epochs argument.
-   The steps_per_epoch argument sets the number of steps per epoch, while the validation_steps argument sets the number of validation steps.
-   Finally, the epochs argument specifies how many batches of data to use for training and validation.
-   The callbacks argument defines two callbacks: early and late.
-   The early callback is called when the first batch of data has been processed, while the late callback is called when all batches have been processed.
-   The verbose flag determines whether or not output information about each step in processing will

In [ ]:
%%time
history = model.fit_generator(generator=train_generator,  
                                    steps_per_epoch=70,
                                    validation_data=valid_generator, 
                                    validation_steps=2,
                                    epochs=nb_epochs,
                                    callbacks = [early],
                                    verbose=2,
                                    workers=4,
                                    use_multiprocessing=True)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['accuracy', 'val_accuracy']].plot()

This code is training a deep learning model using the TensorFlow library. Let's go through the code step by step:

1. The `scheduler` function is defined. This function takes two arguments: `epoch` and `lr` (learning rate). It is used as a callback function to adjust the learning rate during training. If the epoch is less than 10, it returns the current learning rate unchanged. Otherwise, it returns the current learning rate multiplied by the exponential function `tf.math.exp(-0.1)`.

2. The `get_model` function is defined. This function creates a ResNet50V2 model with pre-trained weights from the ImageNet dataset. The model is configured to exclude the top layer, use average pooling, and take input images of size `(img_size, img_size, 3)`. The output of the base model is then passed through a Dense layer with `nb_classes` units and softmax activation. The function returns the complete model.

3. The `model` variable is assigned the result of calling the `get_model` function. This creates the model that will be trained.

4. The `model` is compiled using the Adam optimizer with a learning rate of `lr` and a decay rate of `1e-6`. The loss function is set to `'categorical_crossentropy'` and the metric to track during training is `'accuracy'`.

5. An `EarlyStopping` callback is created. This callback monitors the validation loss and stops training if the loss does not improve for `patience` number of epochs (in this case, 7). The `min_delta` parameter is set to 0, meaning any improvement in the loss will be considered as an improvement.

6. A `LearningRateScheduler` callback is created using the `scheduler` function defined earlier. This callback adjusts the learning rate during training based on the current epoch.

7. The `model.fit_generator` function is called to start the training process. This function takes several arguments:
   - `generator`: The training data generator.
   - `steps_per_epoch`: The number of steps (batches) to take per epoch.
   - `validation_data`: The validation data generator.
   - `validation_steps`: The number of steps (batches) to take for validation.
   - `epochs`: The number of epochs to train for.
   - `callbacks`: A list of callbacks to use during training (in this case, `early` and `callback`).
   - `verbose`: The verbosity mode (0 = silent, 1 = progress bar, 2 = one line per epoch).
   - `workers`: The number of worker processes to use for data loading.
   - `use_multiprocessing`: Whether to use multiprocessing for data loading.

8. The `history` variable is assigned the result of the `model.fit_generator` function. This contains information about the training process, such as the loss and accuracy values at each epoch.

Overall, this code sets up a deep learning model, compiles it with the specified optimizer and loss function, and trains it using the provided data generators and callbacks. The learning rate is adjusted during training using the `LearningRateScheduler` callback, and early stopping is used to prevent overfitting.

In [ ]:
%%time
import tensorflow as tf
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

def get_model():
    K.clear_session()
    base_model =  ResNet50V2(weights='imagenet', include_top=False, pooling='avg', input_shape=(img_size, img_size, 3))
    x = base_model.output
    predictions = Dense(nb_classes, activation="softmax")(x)
    return Model(inputs=base_model.input, outputs=predictions)

model = get_model()
model.compile(optimizers.Adam(lr=lr, decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])

early = EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=1, mode='auto')
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

history = model.fit_generator(generator=train_generator,  
                                    steps_per_epoch=70,
                                    validation_data=valid_generator, 
                                    validation_steps=2,
                                    epochs=nb_epochs,
                                    callbacks = [early,callback],
                                    verbose=2,
                                    workers=4,
                                    use_multiprocessing=True)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['accuracy', 'val_accuracy']].plot()

In [ ]:
import gc
del train_datagen, train_generator
gc.collect()

### Prediction

In [ ]:
sam_sub_df = pd.read_csv('../input/iwildcam-2020-fgvc7/sample_submission.csv')
sam_sub_df["file_name"] = sam_sub_df["Id"].map(lambda str : str + ".jpg")
sam_sub_df.head()

In [ ]:
%%time

test_generator = test_datagen.flow_from_dataframe(      
    
        dataframe=sam_sub_df,    
    
        directory = "../input/iwildcam-2020-fgvc7/test",    
        x_col="file_name",
        target_size = (img_size,img_size),
        batch_size = 1,
        classes = [ str(i) for i in range(nb_classes)],
        shuffle = False,
        class_mode = None
        )

In [ ]:
%%time
test_generator.reset()
predict=model.predict_generator(test_generator, steps = len(test_generator.filenames))
predicted_class_indices=np.argmax(predict,axis=1)
sam_sub_df["Category"] = predicted_class_indices
sam_sub_df = sam_sub_df.loc[:,["Id", "Category"]]
sam_sub_df.to_csv("submission_2.csv",index=False)

The orginal training set contains pictures of many animals at one time. I thought they have a bad impact to learning. By eliminating these, it seems that learning has converged well. Instead, the predictions are not very good. As one of the improvement plans I think, it may be effective to cut out the part of data where the animal is reflected.